In [1]:
import numpy as np
import pandas as pd

# Pre-processing and setup functions
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.utils import to_categorical

# Algorithm
from sklearn.ensemble import GradientBoostingClassifier

# Report and model validation
from sklearn.metrics import classification_report

# Model persistence
from joblib import dump, load

pd.set_option('display.max_columns', 500)

In [2]:
exo2 = pd.read_csv('transformed_features.csv')

In [3]:
exo2.sample(10)

,Unnamed: 0,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,koi_time0bk_err2,koi_impact,koi_impact_err1,koi_impact_err2,koi_duration,koi_duration_err1,koi_duration_err2,koi_depth,koi_depth_err1,koi_depth_err2,koi_prad,koi_prad_err1,koi_prad_err2,koi_teq,koi_insol,koi_insol_err1,koi_insol_err2,koi_model_snr,koi_tce_plnt_num,koi_steff,koi_steff_err1,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag,log_koi_period,log_koi_period_err1,log_koi_time0bk,log_koi_time0bk_err1,log_koi_impact,log_koi_impact_err1,log_koi_duration,log_koi_duration_err1,log_koi_depth,log_koi_depth_err1,log_koi_prad,log_koi_prad_err1,log_koi_teq,log_koi_insol,log_koi_insol_err1,log_koi_model_snr,log_koi_tce_plnt_num,log_koi_steff,log_koi_steff_err1,log_koi_slogg,log_koi_slogg_err1,log_koi_srad,log_koi_srad_err1,log_koi_srad_err2,log_ra,log_dec,log_koi_kepmag
87,87,CONFIRMED,0,0,0,0,3.525633,1.500000e-07,-1.500000e-07,170.047995,0.000033,-0.000033,0.120,0.006,-0.084,2.59959,0.00079,-0.00079,22335.0,13.1,-13.1,11.58,0.90,-0.48,975,213.12,50.79,-29.54,1996.7,1,5150,82,-82,4.572,0.020,-0.064,0.793,0.062,-0.033,290.58173,40.577400,15.224,1.260060,-15.712631,5.136081,-10.309953,-2.111965,-4.961845,0.955354,-6.325540,10.013910,2.572689,2.449279,-0.104250,6.882437,5.361855,3.927719,7.599251,0.0,8.546752,4.406731,1.519951,-3.863233,-0.231932,-2.764621,-3.442019,5.671885,3.703211,2.722873
1238,1238,CONFIRMED,0,0,0,0,37.815416,2.667000e-04,-2.667000e-04,141.103460,0.005680,-0.005680,0.533,0.399,-0.316,4.66200,0.16200,-0.16200,286.4,14.2,-14.2,1.21,0.08,-0.04,401,6.09,1.38,-0.91,22.4,1,4856,86,-106,4.624,0.012,-0.048,0.705,0.049,-0.025,297.29242,49.981628,12.879,3.632717,-8.229386,4.949493,-5.170804,-0.627359,-0.916291,1.539445,-1.814005,5.657389,2.653312,0.190620,-2.513306,5.993961,1.806648,0.322808,3.109061,0.0,8.487970,4.454359,1.531260,-4.342806,-0.349557,-2.995732,-3.729701,5.694716,3.911655,2.555598
4744,4744,FALSE POSITIVE,0,1,1,1,0.745087,2.300000e-06,-2.300000e-06,131.826550,0.002940,-0.002940,0.626,0.335,-0.625,4.29000,1.70000,-1.70000,270.0,31.8,-31.8,1.29,0.20,-0.10,1654,1775.23,892.86,-432.66,61.4,1,5325,143,-159,4.607,0.032,-0.104,0.750,0.122,-0.057,289.16986,48.066780,15.489,-0.294255,-12.982601,4.881487,-5.829346,-0.466809,-1.090644,1.456287,0.531216,5.598422,3.459498,0.254642,-1.604450,7.410952,7.481685,6.794431,4.117410,0.0,8.580168,4.962852,1.527577,-3.411248,-0.287682,-2.095571,-2.882404,5.667014,3.872591,2.740130
5042,5042,FALSE POSITIVE,1,0,0,0,1.444486,6.910000e-06,-6.910000e-06,132.310960,0.003890,-0.003890,0.305,0.165,-0.305,1.43700,0.12800,-0.12800,980.7,98.4,-98.4,1.73,0.12,-0.14,1051,290.01,84.76,-72.08,12.0,1,4569,137,-137,4.686,0.054,-0.031,0.559,0.040,-0.044,291.48495,45.545670,15.356,0.367754,-11.882541,4.885155,-5.549346,-1.184170,-1.795767,0.362558,-2.047943,6.888267,4.589051,0.548121,-2.111965,6.957497,5.669915,4.439836,2.484907,0.0,8.427050,4.919988,1.544579,-2.900422,-0.581606,-3.194183,-3.146555,5.674988,3.818716,2.731506
5818,5818,FALSE POSITIVE,0,0,0,1,1.752516,3.350000e-05,-3.350000e-05,132.754400,0.024200,-0.024200,0.738,0.228,-0.493,3.48900,0.53400,-0.53400,125.9,10.0,-10.0,1.02,0.28,-0.08,1447,1035.27,814.93,-256.53,14.2,1,5952,160,-160,4.554,0.033,-0.176,0.844,0.226,-0.071,299.16608,41.863049,13.978,0.561052,-10.303965,4.888501,-3.721403,-0.302457,-1.474033,1.249615,-0.625489,4.835488,2.302685,0.019803,-1.269401,7.277248,6.942418,6.703103,2.653242,0.0,8.691483,5.075180,1.516006,-3.381395,-0.169603,-1.482805,-2.659260,5.700999,3.734404,2.637485
2596,2596,FALSE POSITIVE,0,0,1,0,1.012672,4.350000e-07,-4.350000e-07,170.189232,0.000340,-0.000340,0.957,0.001,-0.002,3.05120,0.02650,-0.02650,1197.6,9.4,-9.4,5.89,1.83,-0.99,2337,7026.31,6250.59,-2820.42,237.4,1,6663,160,-240,4.275,0.105,-0.195,1.338,0.416,-0.224,298.45319,45.746101,13.788,0.012592,-14.647920,5.136911,-7.986565,-0.0

In [4]:
y = exo2['koi_disposition']
exo2_features = exo2.drop(['koi_disposition', 'Unnamed: 0'], axis=1)

In [5]:
scaler = MinMaxScaler()
scaled_features = scaler.fit_transform(exo2_features)

C:\Users\TPCal\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:334: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [6]:
X_train, X_test, y_train, y_test = train_test_split(scaled_features, y, random_state=42, stratify=y)

In [7]:
gbm_search = GradientBoostingClassifier()

gbm_grid = {
    'learning_rate': [0.1, 0.01, 0.001],
    'n_estimators': [25, 50, 100, 200, 400],
    'max_depth': [1, 2, 3, 4, 5]
}

grid = GridSearchCV(
    gbm_search,
    gbm_grid,
    scoring='accuracy',
    cv=5,
    verbose=3,
    n_jobs=-1
)

In [8]:
grid.fit(X_train, y_train)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Fitting 5 folds for each of 75 candidates, totalling 375 fits


[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:   11.9s
[Parallel(n_jobs=-1)]: Done 112 tasks      | elapsed:  3.4min
[Parallel(n_jobs=-1)]: Done 272 tasks      | elapsed: 10.0min
[Parallel(n_jobs=-1)]: Done 375 out of 375 | elapsed: 15.1min finished


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_sampl...      subsample=1.0, tol=0.0001, validation_fraction=0.1,
              verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'learning_rate': [0.1, 0.01, 0.001], 'n_estimators': [25, 50, 100, 200, 400], 'max_depth': [1, 2, 3, 4, 5]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='accuracy', verbose=3)

In [9]:
print(grid.best_params_)

{'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 200}


In [10]:
print(grid.best_score_)

0.9015830631317948


In [11]:
predictions = grid.predict(X_test)
print(classification_report(y_test, predictions))

                precision    recall  f1-score   support

     CANDIDATE       0.84      0.81      0.82       422
     CONFIRMED       0.82      0.84      0.83       450
FALSE POSITIVE       0.99      1.00      0.99       876

     micro avg       0.91      0.91      0.91      1748
     macro avg       0.88      0.88      0.88      1748
  weighted avg       0.91      0.91      0.91      1748



In [12]:
dump(grid, 'GBM.joblib')

['GBM.joblib']